In [2]:
import zipfile
import os
# Path to the zip file

# Directory to extract the files to
extract_to_directory = './data/'

# Unzipping the file
for file in os.listdir('./data'):
    with zipfile.ZipFile(os.path.join('./data/', file), 'r') as zip_ref:
        zip_ref.extractall(extract_to_directory)

print("File unzipped successfully!")

BadZipFile: File is not a zip file

In [8]:
import pandas as pd
import numpy as np
#garbage collector
import gc
gc.enable()

In [2]:
#contains information about product aisles
aisles = pd.read_csv('data/aisles.csv')
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [3]:
#contains information about product departments
dept = pd.read_csv('data/departments.csv')
dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [ ]:
order_products_prior = pd.read_csv('data/order_products__prior.csv')
order_products_prior
#Here same number for order_id represents multiple products ordered in that single order?

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [5]:
order_products_train = pd.read_csv('data/order_products__train.csv')
order_products_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [ ]:
orders = pd.read_csv('data/orders.csv')
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [ ]:
# contains information about the product and in which aisle and which department it is in
products = pd.read_csv('data/products.csv')
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [11]:
# Chaning the eval_set column in orders dataset
orders['eval_set'] = orders['eval_set'].replace({'prior' : 0, 'train' : 1, 'test': 2})
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(30)
#Here the intuition for filling the NA values with 30 is 30 days represents approximately one month. If a customer has no prior order history, it might be reasonable to assume that their next order will occur in about a month.


In [12]:
#Merging orders and priors datasets
orders_prior = pd.merge(orders, order_products_prior, on = 'order_id', how = 'inner')
orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,0,1,2,8,30.0,196,1,0
1,2539329,1,0,1,2,8,30.0,14084,2,0
2,2539329,1,0,1,2,8,30.0,12427,3,0
3,2539329,1,0,1,2,8,30.0,26088,4,0
4,2539329,1,0,1,2,8,30.0,26405,5,0


In [13]:
#deleting prior dataset
del order_products_prior
gc.collect()

502

Doubts
If order_id is unique identifier for each order, oka user suppose user_id = 1 multiple times order chesthe appudu each time different order ids generate avtai kada, so appudu order_id ane unknown number meeda manam predictions ela generate chestam?